# Object Detection using TAO RetinaNet

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://developer.nvidia.com/sites/default/files/akamai/embedded-transfer-learning-toolkit-software-stack-1200x670px.png" width="1080">

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 RetinaNet model on the KITTI dataset
* Prune the trained retinanet model
* Retrain the pruned model to recover lost accuracy
* Export the pruned model
* Quantize the pruned model using QAT
* Run Inference on the trained model
* Export the pruned, quantized and retrained model to a .etlt file for deployment to DeepStream
* Run inference on the exported. etlt model to verify deployment using TensorRT

## Table of Contents

This notebook shows an example usecase of RetinaNet object detection using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Install the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2) <br>
    2.1 [Download the dataset](#head-2-1) <br>
    2.2 [Validate the downloaded dataset](#head-2-2) <br>
    2.3 [Generate tfrecords from kitti format dataset](#head-2-3) <br>
    2.4 [Download pre-trained model](#head-2-4) <br>
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Prune trained models](#head-6)
7. [Retrain pruned models](#head-7)
8. [Evaluate retrained model](#head-8)
9. [Visualize inferences](#head-9)
10. [Model Export](#head-10)
11. [Verify deployed model](#head-11)

## 1. Set up env variables <a class="anchor" id="head-0"></a>

When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/retinanet`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

In [1]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variable with your key.")
%env KEY=YOUR_KEY
%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/retinanet
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/retinanet

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/retinanet
os.environ["LOCAL_PROJECT_DIR"] = '/home/ubuntu/cv_samples_v1.2.0/retinanet'
os.environ["LOCAL_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data")
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "retinanet")

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tao-experiments/retinanet/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

Please replace the variable with your key.
env: KEY=YOUR_KEY
env: USER_EXPERIMENT_DIR=/workspace/tao-experiments/retinanet
env: DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data
env: SPECS_DIR=/workspace/tao-experiments/retinanet/specs
total 28
-rw-rw-r-- 1 ubuntu ubuntu 1880 Aug 24 23:51 retinanet_train_resnet18_kitti_seq.txt
-rw-rw-r-- 1 ubuntu ubuntu 1884 Aug 24 23:51 retinanet_retrain_resnet18_kitti_seq.txt
-rw-rw-r-- 1 ubuntu ubuntu 1914 Aug 24 23:51 retinanet_retrain_resnet18_kitti.txt
-rw-rw-r-- 1 ubuntu ubuntu  608 Oct 22 14:09 retinanet_tfrecords_kitti_train.txt
-rw-rw-r-- 1 ubuntu ubuntu  604 Oct 22 15:21 retinanet_tfrecords_real_kitti_train.txt
-rw-rw-r-- 1 ubuntu ubuntu 1856 Oct 23 06:28 retinanet_train_fine_resnet18_kitti.txt
-rw-rw-r-- 1 ubuntu ubuntu 1874 Nov 11 12:22 retinanet_train_resnet18_kitti.txt


The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from in and out of the docker. For more information please refer to the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO docker.

```json
drive_map = {
    "Mounts": [
            # Mapping the data directory
            {
                "source": os.environ["LOCAL_PROJECT_DIR"],
                "destination": "/workspace/tao-experiments"
            },
            # Mapping the specs directory.
            {
                "source": os.environ["LOCAL_SPECS_DIR"],
                "destination": os.environ["SPECS_DIR"]
            },
        ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    }
}
```

In [2]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [3]:
!cat ~/.tao_mounts.json

{
    "Mounts": [
        {
            "source": "/home/ubuntu/cv_samples_v1.2.0/retinanet",
            "destination": "/workspace/tao-experiments"
        },
        {
            "source": "/home/ubuntu/cv_samples_v1.2.0/retinanet/specs",
            "destination": "/workspace/tao-experiments/retinanet/specs"
        }
    ]
}

In [5]:
# View the versions of the TAO launcher
!tao info

Configuration of the TAO Toolkit Instance
dockers: ['nvidia/tao/tao-toolkit-tf', 'nvidia/tao/tao-toolkit-pyt', 'nvidia/tao/tao-toolkit-lm']
format_version: 1.0
toolkit_version: 3.21.08
published_date: 08/17/2021


 ## 2. Download and Prepare dataset

Create a data folder inside the root directory and download the synthetic and real datasets from the provided links.

Dataset is already formatted into train and test for both synthetic and real cases. 

We have 2 scenarios in the dataset i.e dataset with less complex background and dataset with complex background.


Following directory structure will be there according to the spec files. 

├── real_test
├── real_train_20
├── synthetic_test
├── synthetic_train


## 3 Prepare tfrecords from kitti format dataset <a class="anchor" id="head-2-3"></a>

* Update the tfrecords spec file to take in your kitti format dataset
* Create the tfrecords using the dataset_convert 
* TFRecords only need to be generated once.

In [6]:
# verify
!ls -l $LOCAL_DATA_DIR

total 20
drwxrwxr-x 6 ubuntu ubuntu 4096 Oct 23 10:53 real_test
drwxrwxr-x 4 ubuntu ubuntu 4096 Oct 17 01:41 real_train_20
drwxrwxr-x 4 ubuntu ubuntu 4096 Oct 16 18:47 synthetic_test
drwxrwxr-x 4 ubuntu ubuntu 4096 Oct 16 18:20 synthetic_train
drwxrwxr-x 4 ubuntu ubuntu 4096 Oct 16 18:20 synthetic_train1


In [8]:
print("TFRecords conversion spec file:")
!cat $LOCAL_SPECS_DIR/retinanet_tfrecords_kitti_train.txt

TFRecords conversion spec file:
kitti_config {
  root_directory_path: "/workspace/tao-experiments/data/"
  image_dir_name: "synthetic_train/images"
  label_dir_name: "synthetic_train/labels"
  image_extension: ".png"
  partition_mode: "random"
  num_partitions: 2
  val_split: 0
  num_shards: 10
}
image_directory_path: "/workspace/tao-experiments/data/"
  target_class_mapping {
      key: "screw1"
      value: "screw1"
  }
  target_class_mapping {
      key: "screw3"
      value: "screw3"
  }
  target_class_mapping {
      key: "screw6"
      value: "screw6"
  }
  target_class_mapping {
      key: "screw8"
      value: "screw8"
  }



In [9]:
# Creating a new directory for the output tfrecords dump.
print("Converting the training set to TFRecords.")
!mkdir -p $LOCAL_DATA_DIR/tfrecords && rm -rf $LOCAL_DATA_DIR/tfrecords/*
!tao retinanet dataset_convert \
               -d $SPECS_DIR/retinanet_tfrecords_kitti_train.txt \
               -o $DATA_DOWNLOAD_DIR/tfrecords/kitti_train

Converting the training set to TFRecords.
2021-11-11 12:26:43,180 [INFO] root: Registry: ['nvcr.io']
2021-11-11 12:26:43,259 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/ubuntu/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.
2021-11-11 12:26:50,532 - iva.detectnet_v2.dataio.build_converter - INFO - Instantiating a kitti converter
2021-11-11 12:26:50,556 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Num images in
Train: 10661	Val: 0
2021-11-11 12:26:50,556 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Skipped validation data...
2021-11-11 12:26:50,561 - iva.detectnet_v2.dataio.dataset_converter_lib - INFO - Writing partition 0, shard 0

2021-11-11 12:26:50,561 - tenso

In [10]:
# Creating a new directory for the output tfrecords dump.
print("Converting the training set to TFRecords.")
# !mkdir -p $LOCAL_DATA_DIR/tfrecords && rm -rf $LOCAL_DATA_DIR/tfrecords/*
!tao retinanet dataset_convert \
               -d $SPECS_DIR/retinanet_tfrecords_real_kitti_train.txt \
               -o $DATA_DOWNLOAD_DIR/tfrecords/kitti_train_20

Converting the training set to TFRecords.
2021-11-11 12:27:47,832 [INFO] root: Registry: ['nvcr.io']
2021-11-11 12:27:47,961 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/ubuntu/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
Using TensorFlow backend.
Using TensorFlow backend.
2021-11-11 12:27:55,079 - iva.detectnet_v2.dataio.build_converter - INFO - Instantiating a kitti converter
2021-11-11 12:27:55,080 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Num images in
Train: 195	Val: 0
2021-11-11 12:27:55,081 - iva.detectnet_v2.dataio.kitti_converter_lib - INFO - Skipped validation data...
2021-11-11 12:27:55,081 - iva.detectnet_v2.dataio.dataset_converter_lib - INFO - Writing partition 0, shard 0

2021-11-11 12:27:55,081 - tensorf

In [11]:
!ls -rlt $LOCAL_DATA_DIR/tfrecords/

total 3729960
-rw-r--r-- 1 root root 376880632 Nov 11 12:26 kitti_train-fold-000-of-002-shard-00000-of-00010
-rw-r--r-- 1 root root 381808035 Nov 11 12:26 kitti_train-fold-000-of-002-shard-00001-of-00010
-rw-r--r-- 1 root root 377996527 Nov 11 12:27 kitti_train-fold-000-of-002-shard-00002-of-00010
-rw-r--r-- 1 root root 379597772 Nov 11 12:27 kitti_train-fold-000-of-002-shard-00003-of-00010
-rw-r--r-- 1 root root 384238153 Nov 11 12:27 kitti_train-fold-000-of-002-shard-00004-of-00010
-rw-r--r-- 1 root root 380008862 Nov 11 12:27 kitti_train-fold-000-of-002-shard-00005-of-00010
-rw-r--r-- 1 root root 379196557 Nov 11 12:27 kitti_train-fold-000-of-002-shard-00006-of-00010
-rw-r--r-- 1 root root 382253015 Nov 11 12:27 kitti_train-fold-000-of-002-shard-00007-of-00010
-rw-r--r-- 1 root root 385609901 Nov 11 12:27 kitti_train-fold-000-of-002-shard-00008-of-00010
-rw-r--r-- 1 root root 382992608 Nov 11 12:27 kitti_train-fold-000-of-002-shard-00009-of-00010
-rw-r--r-- 1 root root     17803 Nov

 ## 4 NGC CLI Instalaltion & Download pre-trained model <a class="anchor" id="head-2-4"></a>


We will use NGC CLI to get the pre-trained models. For more details, go to [ngc.nvidia.com](ngc.nvidia.com) and click the SETUP on the navigation bar.

In [12]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2021-11-11 12:28:07--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 13.249.38.69, 13.249.38.66, 13.249.38.126, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|13.249.38.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25122135 (24M) [application/zip]
Saving to: ‘/home/ubuntu/cv_samples_v1.2.0/retinanet/ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  23.96M   102MB/s    in 0.2s    

2021-11-11 12:28:07 (102 MB/s) - ‘/home/ubuntu/cv_samples_v1.2.0/retinanet/ngccli/ngccli_cat_linux.zip’ saved [25122135/25122135]

Archive:  /home/ubuntu/cv_samples_v1.2.0/retinanet/ngccli/ngccli_cat_linux.zip
  inflating: /home/ubuntu/cv_samples_v1.2.0/retinanet/ngccli/ngc  
 extracting: /home/ubuntu/cv_samples_v1.2.0/retinanet/ngccli/ngc.md5  


In [13]:
!ngc registry model list nvidia/tao/pretrained_object_detection:*

+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| vgg19 | 77.56 | 80    | 1     | V100  | 153.7 | 153.7 | UPLOA | Aug   |
|       |       |       |       |       |       | 2 MB  | D_COM | 18,   |
|       |       |       |       |       |       |       | PLETE | 2021  |
| vgg16 | 77.17 | 80    | 1     | V100  | 113.2 | 113.1 | UPLOA | Aug   |
|       |       |       |       |       |       | 6 MB  | D_COM | 18,   |
|       |       |       |       |       |       |       | PLETE | 2021  |
| squee | 65.13 | 80    | 1     | V100  | 6.5   | 6.46  | UPLOA | Aug   |
| zenet |       |       |       |     

In [14]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/

In [15]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet18 \
                    --dest $LOCAL_EXPERIMENT_DIR/pretrained_resnet18

Downloaded 82.38 MB in 6s, Download speed: 13.71 MB/s               
----------------------------------------------------
Transfer id: pretrained_object_detection_vresnet18 Download status: Completed.
Downloaded local path: /home/ubuntu/cv_samples_v1.2.0/retinanet/retinanet/pretrained_resnet18/pretrained_object_detection_vresnet18-4
Total files downloaded: 1 
Total downloaded size: 82.38 MB
Started at: 2021-11-11 12:28:13.086924
Completed at: 2021-11-11 12:28:19.099040
Duration taken: 6s
----------------------------------------------------


In [16]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18

Check that model is downloaded into dir.
total 91100
-rw------- 1 ubuntu ubuntu 93278448 Oct 22 14:12 resnet_18.hdf5


 ## 5. Provide training specification <a class="anchor" id="head-3"></a>

In [17]:
!cat $LOCAL_SPECS_DIR/retinanet_train_resnet18_kitti.txt

random_seed: 42
retinanet_config {
  aspect_ratios_global: "[1.0, 2.0, 0.5]"
  scales: "[0.045, 0.09, 0.2, 0.4, 0.55, 0.7]"
  two_boxes_for_ar1: false
  clip_boxes: false
  loss_loc_weight: 0.8
  focal_loss_alpha: 0.25
  focal_loss_gamma: 2.0
  variances: "[0.1, 0.1, 0.2, 0.2]"
  arch: "resnet"
  nlayers: 18
  n_kernels: 1
  n_anchor_levels: 1
  feature_size: 256
  freeze_bn: False
  freeze_blocks: 0
}
training_config {
  enable_qat: False
  pretrain_model_path: "/workspace/tao-experiments/retinanet/pretrained_resnet18/pretrained_object_detection_vresnet18/resnet_18.hdf5"
  batch_size_per_gpu: 8
  num_epochs: 100
  n_workers: 2
  checkpoint_interval: 10
  learning_rate {
    soft_start_annealing_schedule {
      min_learning_rate: 4e-5
      max_learning_rate: 1.5e-2
      soft_start: 0.1
      annealing: 0.3
    }
  }
  regularizer {
    type: L1
    weight: 2e-5
  }
  optimizer {
    sgd {
      momentum: 0.9
      nesterov: True
    }
  }
}
eval_config {
  validation_period_during_t

In [18]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned

 ## 6. Run TAO training on Synthetic Dataset<a class="anchor" id="head-4"></a>
 * Provide the sample spec file for training.

In [19]:
!sed -i "s|YOUR_PRETRAINED_MODEL|$USER_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18/resnet_18.hdf5|g" $LOCAL_SPECS_DIR/retinanet_train_resnet18_kitti.txt
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!tao retinanet train -e $SPECS_DIR/retinanet_train_resnet18_kitti.txt \
                     -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
                     -k $KEY \
                     --gpus 1 --use_amp > out_resnet18_synth_amp16.log

To run with multigpu, please change --gpus based on the number of available GPUs in your machine.
2021-11-11 12:28:20,247 [INFO] root: Registry: ['nvcr.io']
2021-11-11 12:28:20,328 [WARNING] tlt.components.docker_handler.docker_handler: 
Docker will run the commands as root. If you would like to retain your
local host permissions, please add the "user":"UID:GID" in the
DockerOptions portion of the "/home/ubuntu/.tao_mounts.json" file. You can obtain your
users UID and GID by using the "id -u" and "id -g" commands on the
terminal.
2021-11-11 12:30:28,725 [INFO] tlt.components.docker_handler.docker_handler: Stopping container.


In [20]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned_fine

 ## 7. Run TAO training or fine tuning on Real Dataset<a class="anchor" id="head-4"></a>
 * Provide the sample spec file for training.

In [ ]:
!sed -i "s|YOUR_PRETRAINED_MODEL|$USER_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18/resnet_18.hdf5|g" $LOCAL_SPECS_DIR/retinanet_train_resnet18_kitti.txt
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!tao retinanet train -e $SPECS_DIR/retinanet_train_fine_resnet18_kitti.txt \
                     -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned_fine \
                     -k $KEY \
                     --gpus 1 --use_amp > out_resnet18_synth_fine_amp16.log

## 8. Evaluate trained models <a class="anchor" id="head-5"></a>

In [ ]:
!tao retinanet evaluate -e $SPECS_DIR/retinanet_train_fine_resnet18_kitti.txt \
                        -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned_fine/weights/retinanet_resnet18_epoch_100.tlt \
                        -k $KEY

## 9. Visualize inferences <a class="anchor" id="head-9"></a>
In this section, we run the tlt-infer tool to generate inferences on the trained models and visualize the results.

In [ ]:
# Running inference for detection on n images
!tao retinanet inference -i $DATA_DOWNLOAD_DIR/real_test/images \
                         -o $USER_EXPERIMENT_DIR/retinanet_infer_images \
                         -e $SPECS_DIR/retinanet_train_fine_resnet18_kitti.txt \
                         -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned_fine/weights/retinanet_resnet18_epoch_100.tlt \
                         -l $USER_EXPERIMENT_DIR/retinanet_infer_labels \
                         -k $KEY

The `inference` tool produces two outputs. 
1. Overlain images in `$LOCAL_EXPERIMENT_DIR/retinanet_annotated_images`
2. Frame by frame bbox labels in kitti format located in `$LOCAL_EXPERIMENT_DIR/retinanet_infer_labels`

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=1, num_images=1):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'retinanet_infer_images' # relative path from $LOCAL_EXPERIMENT_DIR.
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)